In [ ]:
import cv2
import numpy as np
import tensorflow as tf

# Load cascaded CNN classifiers for eye detection
eye_net1 = cv2.dnn.readNetFromTensorflow('models/eye_net1.pb')
eye_net2 = cv2.dnn.readNetFromTensorflow('models/eye_net2.pb')

# Initialize video capture object
cap = cv2.VideoCapture(0)

while True:
    # Capture frame from camera
    ret, frame = cap.read()

    # Convert frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Define regions of interest for left and right eyes
    left_eye_roi = gray[110:180, 150:220]
    right_eye_roi = gray[110:180, 260:330]

    # Preprocess eye ROIs
    left_eye_roi = cv2.resize(left_eye_roi, (24, 24))
    right_eye_roi = cv2.resize(right_eye_roi, (24, 24))
    left_eye_roi = cv2.equalizeHist(left_eye_roi)
    right_eye_roi = cv2.equalizeHist(right_eye_roi)
    left_eye_roi = left_eye_roi.astype('float32') / 255.0
    right_eye_roi = right_eye_roi.astype('float32') / 255.0
    left_eye_roi = np.expand_dims(left_eye_roi, axis=2)
    right_eye_roi = np.expand_dims(right_eye_roi, axis=2)

    # Predict eye states using cascaded CNNs
    left_eye_state = 'open'
    right_eye_state = 'open'
    for net in [eye_net1, eye_net2]:
        left_eye_pred = net.forward(left_eye_roi[np.newaxis, ...])
        right_eye_pred = net.forward(right_eye_roi[np.newaxis, ...])
        left_eye_state = 'closed' if np.argmax(left_eye_pred) == 0 else 'open'
        right_eye_state = 'closed' if np.argmax(right_eye_pred) == 0 else 'open'

    # Draw eye state on the frame
    cv2.putText(frame, left_eye_state, (150, 100), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
    cv2.putText(frame, right_eye_state, (260, 100), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

    # Display the frame
    cv2.imshow('Eye Detector', frame)

    # Exit if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture object and destroy all windows
cap.release()
cv2.destroyAllWindows()
